# Introduction
In this work, I decided to write my own implementation of the linear regression algorithm and compare its performance with the algorithm implemented in the Scikit-learn library.
## Importing the required libraries

In [1]:
import pandas as pd
import random
from typing import List

## Data loading
The data frame will not be divided into train and test parts, since I'm only interested in the relative accuracy of the algorithms.

In [2]:
df = pd.read_csv('/kaggle/input/startup-logistic-regression/50_Startups.csv')
df1 = df.copy().drop('State', axis = 1)
df1.head()

,R&D Spend,Administration,Marketing Spend,Profit
0,165349.20,136897.80,471784.10,192261.83
1,162597.70,151377.59,443898.53,191792.06
2,153441.51,101145.55,407934.54,191050.39
3,144372.41,118671.85,383199.62,182901.99
4,142107.34,91391.77,366168.42,166187.94


# Algorithm implementation
The target variable is assumed to be the last column of the data frame.

In [3]:
#Linear function
def linear_func(x: List[float], coeff: List[float]) -> float:
    assert len(coeff) - len(x) == 1
    return coeff[0] + sum([x_i * coeff_i for x_i, coeff_i in zip(x, coeff[1:])])

#A function that calculates new coefficients based on gradient descent
def new_coeff(prev_coeff: List[float], gradient: List[float], step: float) -> List[float]:
    assert len(prev_coeff) == len(gradient)
    grad_step = [step * grad for grad in gradient]
    return [coeff - grad for coeff, grad in zip(prev_coeff, grad_step)]

#Function for calculating the average relative error
def avg_rel_error(y: List[float], y_pred: List[float]) -> float:
    error = [abs(y_i - y_pred_i) / y_i for y_i, y_pred_i in zip(y, y_pred)]
    return sum(error) / len(y)

#Function to fit the data by linear regression. At the output, the function gives the coefficients of the linear model
def linear_regression(df: pd.DataFrame, step: int = 0.000001) -> List[float]:
    rel_error0, rel_error1 = 2, 1
    
    #Assignment of random values to coefficients for subsequent optimization of the sum of squared errors function
    coeff = [random.random() for _ in range(df.shape[1])]

    #The learning process stops when the relative error does not decrease
    while rel_error0 - rel_error1 > 0.00000001:
        rel_error0 = rel_error1
        gradient = []

        #Calculating the gradient of the sum of squared errors function at the current point
        for i in range(len(coeff)):
            if i == 0:
                gradient.append(sum([-2 * (df.iloc[j, -1] - linear_func(df.iloc[j, :-1], coeff)) for j in range(df.shape[0])]))
            else:
                gradient.append(sum([-2 * df.iloc[j, i - 1] * (df.iloc[j, -1] - linear_func(df.iloc[j, :-1], coeff)) for j in range(df.shape[0])]))

        #Calculating the coordinates of a new point using gradient descent
        coeff = new_coeff(coeff, gradient, step)
        
        #Calculation of the average relative error in order to decide whether to continue training or not
        y_pred = [linear_func(df.iloc[k, :-1], coeff) for k in range(df.shape[0])]
        rel_error1 = avg_rel_error(df.iloc[:, -1], y_pred)
    
    return coeff

# Performance comparison
## My algorithm implementation
Learning step was chosen empirically.

In [4]:
random.seed(1)

coeff = linear_regression(df1, 10**-13)
predict = [linear_func(df1.iloc[i, :-1], coeff) for i in range(df1.shape[0])]

error = avg_rel_error(df1.iloc[:, -1], predict)

## Scikit-learn's LinearRegression

In [5]:
from sklearn.linear_model import LinearRegression

df2 = df.copy().drop('State', axis = 1)

sk_model = LinearRegression(fit_intercept = False).fit(df2.iloc[:, :-1], df2.iloc[:, -1])
sk_predict = sk_model.predict(df2.iloc[:, :-1])

sk_error = avg_rel_error(df2.iloc[:, -1], sk_predict)

## Results

In [6]:
print('My algorithm\'s error:', error)
print('Scikit-learn\'s error:', sk_error)

My algorithm's error: 0.14707209291086243
Scikit-learn's error: 0.1470710629123255


The values are almost equal. I think this is a good result.